Goal: Investigate different landmarks on LGI-PPGI

a. Raw Signal Extraction
- Skin extraction: convex hull
- 468 keypoints: patch, landmarks definition from thesis
    - Modified, should it also include the points inside and not just contour ? 
- 28 ROIs --> TODO how to combine them ? 
- Raw RGB Traces

b. HR Estimation 
- rPPG Methods: OMIT, POS, CHROM, LGI
- BVP Signal
- Estimated HR Values
- Evaluation: MAE, PCC + RMSE, SNR

In [32]:
%load_ext autoreload
%autoreload 2

import pyVHR as vhr
import numpy as np
from pyVHR.analysis.pipeline import Pipeline
from pyVHR.plot.visualize import *
import os
import plotly.express as px
from pyVHR.utils.errors import getErrors, printErrors, displayErrors

import constants
import pandas as pd
import pyVHR.analysis.pipelineLandmarks as custom_pipeline

vhr.plot.VisualizeParams.renderer = 'vscode' 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# -- LOAD A DATASET

dataset_name = 'mr_nirp'    
video_DIR, BVP_DIR = constants.get_dataset_paths(dataset_name)

dataset = vhr.datasets.datasetFactory(dataset_name, videodataDIR=video_DIR, BVPdataDIR=BVP_DIR)
allvideo = dataset.videoFilenames

# print the list of video names with the progressive index (idx)
for v in range(len(allvideo)):
  print(v, allvideo[v])

0 D:/datasets_rppg/MR-NIRP_indoor\Subject1_motion_940\Subject1_motion_940\RGB_corrected\Subject1_motion_940.avi
1 D:/datasets_rppg/MR-NIRP_indoor\Subject1_still_940-015\Subject1_still_940\RGB_corrected\Subject1_still_940.avi
2 D:/datasets_rppg/MR-NIRP_indoor\Subject2_motion_940\Subject2_motion_940\RGB_corrected\Subject2_motion_940.avi
3 D:/datasets_rppg/MR-NIRP_indoor\Subject2_still_940-002\Subject2_still_940\RGB_corrected\Subject2_still_940.avi
4 D:/datasets_rppg/MR-NIRP_indoor\Subject3_motion_940\Subject3_motion_940\RGB_corrected\Subject3_motion_940.avi
5 D:/datasets_rppg/MR-NIRP_indoor\Subject3_still_940-012\Subject3_still_940\RGB_corrected\Subject3_still_940.avi
6 D:/datasets_rppg/MR-NIRP_indoor\Subject4_motion_940\Subject4_motion_940\RGB_corrected\Subject4_motion_940.avi
7 D:/datasets_rppg/MR-NIRP_indoor\Subject4_still_940-004\Subject4_still_940\RGB_corrected\Subject4_still_940.avi
8 D:/datasets_rppg/MR-NIRP_indoor\Subject5_still_940-003\Subject5_still_940\RGB_corrected\Subject5_s

In [4]:
# -- PARAMETER SETTING

roi = 'mustache'      # jaw, forehead, cheeks, nose, temple, lip
wsize = 8        # seconds of video processed (with overlapping) for each estimate 
min_len = 2     # minimum number of landmarks tested
patch_size = constants.get_patch_size(dataset_name)
print(f"Patch size for {dataset_name} is {patch_size} ")
print(f"Testing ROI = {roi} with minimum {min_len} landmarks")  

Patch size for mr_nirp is 60 
Testing ROI = mustache with minimum 2 landmarks


# ROI

In [17]:
rois = {
  'forehead': [   
      'lower_medial_forehead','glabella','left_lower_lateral_forehead','right_lower_lateral_forehead'
    ],
 'nose': [
    'upper_nasal_dorsum','lower_nasal_dorsum','left_mid_nasal_sidewall','right_mid_nasal_sidewall','left_lower_nasal_sidewall',
    'right_lower_nasal_sidewall','nasal_tip','soft_triangle','left_ala','right_ala'
  ],
  'cheeks':[
    'left_malar','right_malar', 'left_lower_cheek','right_lower_cheek'
  ],
  'jaw':[
    'left_marionette_fold','right_marionette_fold','chin'
  ],
  'temple':[
    'left_temporal','right_temporal'
  ],
  'mustache':[
    'left_nasolabial_fold','right_nasolabial_fold','left_upper_lip','right_upper_lip','philtrum'
  ],
}

forehead_params = [['left_lower_lateral_forehead', 'right_lower_lateral_forehead'], ['glabella', 'lower_medial_forehead'],  ['left_lower_lateral_forehead', 'lower_medial_forehead', 'right_lower_lateral_forehead'],['glabella', 'left_lower_lateral_forehead', 'right_lower_lateral_forehead'],['glabella', 'left_lower_lateral_forehead', 'lower_medial_forehead', 'right_lower_lateral_forehead']]
cheeks_params =  [['left_malar', 'right_malar'], ['left_lower_cheek', 'right_lower_cheek'], ['left_lower_cheek', 'left_malar', 'right_lower_cheek', 'right_malar']]
jaw_params = [['left_marionette_fold', 'right_marionette_fold'], ['chin', 'left_marionette_fold', 'right_marionette_fold']]
mustache_params = [['left_nasolabial_fold', 'right_nasolabial_fold'], ['left_upper_lip', 'right_upper_lip'], ['left_nasolabial_fold', 'philtrum', 'right_nasolabial_fold'], ['left_upper_lip', 'philtrum', 'right_upper_lip'], ['left_nasolabial_fold', 'left_upper_lip', 'right_nasolabial_fold', 'right_upper_lip'], ['left_nasolabial_fold', 'left_upper_lip', 'philtrum', 'right_nasolabial_fold', 'right_upper_lip']]

In [50]:
roi_combinations = [('forehead', 'cheeks'), ('forehead', 'jaw'), ('jaw', 'cheeks')]
landmarks_dict = dict()
for roi_combination in roi_combinations:
    landmarks_dict[roi_combination] = [ldmk for roi in list(map(rois.get, roi_combination)) for ldmk in roi]
landmarks_list = list(landmarks_dict.values())
print(landmarks_dict)

{('forehead', 'cheeks'): ['lower_medial_forehead', 'glabella', 'left_lower_lateral_forehead', 'right_lower_lateral_forehead', 'left_malar', 'right_malar', 'left_lower_cheek', 'right_lower_cheek'], ('forehead', 'jaw'): ['lower_medial_forehead', 'glabella', 'left_lower_lateral_forehead', 'right_lower_lateral_forehead', 'left_marionette_fold', 'right_marionette_fold', 'chin'], ('jaw', 'cheeks'): ['left_marionette_fold', 'right_marionette_fold', 'chin', 'left_malar', 'right_malar', 'left_lower_cheek', 'right_lower_cheek']}


# Pipeline

In [34]:
class CombineLandmarks:
    def __init__(self, videos, dataset, winsize, patch_size, pipeline, methods=['cupy_CHROM'], verb=False):
        self.dataset = dataset
        self.winsize = winsize
        self.patch_size = patch_size
        self.methods = methods
        self.pipeline = pipeline
        self.res = pd.DataFrame()
        self.verb = verb

        # Load ground truth data
        self.sigGT = []
        self.timesGT = []
        self.bpmGT = []
        self.videoFileName = []
        self.losses = []
        self.load_ground_truth(videos)

    def load_ground_truth(self, videos):
        for videoIdx in videos:
            try: 
                fname = dataset.getSigFilename(videoIdx)
                sigGT = dataset.readSigfile(fname)
                bpmGT, timesGT = sigGT.getBPM(self.winsize)
                self.sigGT.append(sigGT)
                self.bpmGT.append(bpmGT)
                self.timesGT.append(timesGT)
                self.videoFileName.append(dataset.getVideoFilename(videoIdx))
                self.fps = vhr.extraction.get_fps(self.videoFileName[-1]) # assuming they are all the same
            except Exception as e:
                print(f"{videoIdx}: {e}")
                continue
        print('Video name: ', {len(self.videoFileName)}, self.videoFileName)
        print('Video frame rate: ',self.fps)

    def process(self,landmarks):
        losses = []
        for i, videoName in enumerate(self.videoFileName):
            print(videoName)
            res = self.pipeline.run_on_video_multimethods(
                    ldmks_list=landmarks, 
                    videoFileName=self.videoFileName[i], bpmGT=self.bpmGT[i], timesGT=self.timesGT[i], 
                    methods=self.methods, winsize=self.winsize, patch_size=self.patch_size,
                    verb=self.verb
                )
            losses.append(res.dict['RMSE'][0]) # suppose we are minimizing RMSE
            self.res = pd.concat([self.res, res.dataFrame])
        print(f"Total loss for {ldmks_list}: {sum(losses)}")
        self.losses.append(losses)

    def fit(self, landmarks_list):
        for landmarks in landmarks_list:
            print("Processing landmarks: ", landmarks)
            self.process(landmarks)

In [53]:
pl = custom_pipeline.LandmarksPipeline()
# np.arange(0, len(dataset.videoFilenames))
model = CombineLandmarks([0], dataset, wsize, patch_size, pl, methods=['cupy_CHROM'], verb=False)
model.fit([landmarks_list[0]])

Video name:  {1} ['D:/datasets_rppg/MR-NIRP_indoor\\Subject1_motion_940\\Subject1_motion_940\\RGB_corrected\\Subject1_motion_940.avi']
Video frame rate:  30.0
D:/datasets_rppg/MR-NIRP_indoor\Subject1_motion_940\Subject1_motion_940\RGB_corrected\Subject1_motion_940.avi
Total loss for ['left_nasolabial_fold', 'right_nasolabial_fold', 'left_upper_lip', 'right_upper_lip', 'philtrum']: 1.7513122899945066


In [67]:
df['landmarks'] = df['landmarks'].apply(lambda x: tuple(x))
# df['dataset'] = f'{dataset_name}_{roi}'
# if dataset_name == 'lgi_ppgi':
#   df['videoFilename'] = df.videoFilename.apply(lambda x: x.split('\\')[2])
# if dataset_name == 'mr_nirp':
#   df['videoFilename'] = df.videoFilename.apply(lambda x: x.split('\\')[2])
indexes = {}
for v in range(len(allvideo)):
  indexes[allvideo[v].split('\\')[2]] = v
indexes = pd.DataFrame({'videoIdx':list(indexes.values()), 'videoFilename':list(indexes.keys())})
df = df.drop(columns=['sigFilename', 'bpmES_mad', 'videoIdx'])
df.insert(3, 'videoIdx', df['videoFilename'].map(indexes.set_index('videoFilename')['videoIdx']))

from fastdtw import fastdtw
df.insert(11, 'DTW', None)
for row in df.itertuples():
  distance, path = fastdtw(row.bpmGT, row.bpmES)
  df.loc[row.Index, 'DTW'] = [distance]

In [78]:
pd.DataFrame.from_dict(landmarks_dict, orient='index',)


,0,1,2,3,4,5,6,7
"(forehead, cheeks)",lower_medial_forehead,glabella,left_lower_lateral_forehead,right_lower_lateral_forehead,left_malar,right_malar,left_lower_cheek,right_lower_cheek
"(forehead, jaw)",lower_medial_forehead,glabella,left_lower_lateral_forehead,right_lower_lateral_forehead,left_marionette_fold,right_marionette_fold,chin,None
"(jaw, cheeks)",left_marionette_fold,right_marionette_fold,chin,left_malar,right_malar,left_lower_cheek,right_lower_cheek,None
